In [2]:
import pandas as pd
import numpy as np

In [12]:
features=pd.read_csv('./temps.csv').drop(['forecast_noaa','forecast_acc','forecast_under'],axis=1)
features.head()

,year,month,day,week,temp_2,temp_1,average,actual,friend
0,2016,1,1,Fri,45,45,45.6,45,29
1,2016,1,2,Sat,44,45,45.7,44,61
2,2016,1,3,Sun,45,44,45.8,41,56
3,2016,1,4,Mon,44,41,45.9,40,53
4,2016,1,5,Tues,41,40,46.0,44,41


In [25]:
features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 348 entries, 0 to 347
Data columns (total 15 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   year        348 non-null    int64  
 1   month       348 non-null    int64  
 2   day         348 non-null    int64  
 3   temp_2      348 non-null    int64  
 4   temp_1      348 non-null    int64  
 5   average     348 non-null    float64
 6   actual      348 non-null    int64  
 7   friend      348 non-null    int64  
 8   week_Fri    348 non-null    uint8  
 9   week_Mon    348 non-null    uint8  
 10  week_Sat    348 non-null    uint8  
 11  week_Sun    348 non-null    uint8  
 12  week_Thurs  348 non-null    uint8  
 13  week_Tues   348 non-null    uint8  
 14  week_Wed    348 non-null    uint8  
dtypes: float64(1), int64(7), uint8(7)
memory usage: 24.3 KB


In [20]:
features=pd.get_dummies(features)
features.head()

,year,month,day,temp_2,temp_1,average,actual,friend,week_Fri,week_Mon,week_Sat,week_Sun,week_Thurs,week_Tues,week_Wed
0,2016,1,1,45,45,45.6,45,29,1,0,0,0,0,0,0
1,2016,1,2,44,45,45.7,44,61,0,0,1,0,0,0,0
2,2016,1,3,45,44,45.8,41,56,0,0,0,1,0,0,0
3,2016,1,4,44,41,45.9,40,53,0,1,0,0,0,0,0
4,2016,1,5,41,40,46.0,44,41,0,0,0,0,0,1,0


In [21]:
feature_labels=features.columns.values

y=features['actual']
x=np.array(features)

In [27]:
print("Shape of y:",y.shape)
print("Shape of x:",x.shape)

Shape of y: (348,)
Shape of x: (348, 15)


In [29]:
from sklearn.model_selection import train_test_split as tts
xtrain,xtest,ytrain,ytest=tts(x,y,test_size=0.4,random_state=42)

print("Training features shape: ",xtrain.shape)
print("Training labels shape: ",ytrain.shape)
print("Test features shape: ",xtest.shape)
print("Test labels shape: ",ytest.shape)

Training features shape:  (208, 15)
Training labels shape:  (208,)
Test features shape:  (140, 15)
Test labels shape:  (140,)


In [50]:
feature_list = list(features.columns)

# The baseline predictions are the historical averages
#our baseline is the error we would get if we simply predicted the average max temperature for all days.
baseline_preds = xtest[:, feature_list.index('average')] #the second arg gives index of the 'average' column in the array i.e in xtest[:,5] gives list of average temps

# Baseline errors, and display average baseline error
baseline_errors = abs(baseline_preds - ytest)
print('Average baseline error: ', round(np.mean(baseline_errors), 2))

Average baseline error:  4.85


In [51]:
#implement model
from sklearn.ensemble import RandomForestRegressor

#initialise 1000 decision trees
rfr=RandomForestRegressor(1000,random_state=4)

#train model
rfr.fit(xtrain,ytrain)

RandomForestRegressor(n_estimators=1000, random_state=4)

In [75]:
predictions=rfr.predict(xtest)
acc=np.mean(abs(predictions-ytest)/ytest)
# acc=rfr.score(xtest,ytest)
print("Accuracy is: ",(1-acc)*100,"%")

#finding mean absolute error for comparison with baseline
mae=np.mean(abs(predictions-ytest))
print("Mean absolute error: ",round(mae,3),' degrees')

print("Improvement wrt baseline: ",100*abs(mae-np.mean(baseline_errors))/np.mean(baseline_errors),"%")

Accuracy is:  99.63378093119573 %
Mean absolute error:  0.203  degrees
Improvement wrt baseline:  95.8065086143425 %
